In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# %%
import torch
import string

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 14.7 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 14.3 MB/s 
     |████████████████████████████████| 3.3 MB 80.6 MB/s 
     |████████████████████████████████| 895 kB 72.5 MB/s 
     |████████████████████████████████| 636 kB 74.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [ ]:
import pandas as pd
df = pd.read_csv("stockholm/mlm/new_literal.csv")

##BERT

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/experiments'
/content/drive/My Drive/Colab Notebooks/experiments


In [ ]:
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd
data = pd.read_csv("data/moh-x.csv")

# Split to train, val and test
text, test_df = tts(data[["sentence", "arg1", "arg2", "verb", "label"]], random_state=42, test_size=0.1)
text, val = tts(text, random_state=42, test_size=test_df.shape[0])

In [ ]:
text = text["sentence"]

In [ ]:
#text.to_csv(r'mohx.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
with open('data/trofix.txt', 'r') as fp:
    text = fp.read().split('\n')

In [ ]:
text[:5]

['"Just as Reuters and Telerate want to grab a piece of Quotron \'s territory by offering U.S. equities quotes , Quotron is eyeing their turf and mulling ways to offer pricing information on fixed-income instruments and foreign currencies ."',
 '"With its strength in the Old South , the Rocky Mountain and northern plains states and such dependable GOP bastions as Indiana , the party can absorb some losses and still accumulate close to 200 electoral votes ."',
 '"`` We \'ve been plowing a lot of new furrows , \'\' says Richard Burket of Archer-Daniels-Midland Co . , Decatur , Ill ."',
 '"Their ambassador , Oleg Sokolov , speaks fluent English , goes to prayer breakfasts with President Corazon Aquino , and gives smooth interviews to the Philippine press in which he urges the country to kick out the Americans and get friendlier with Moscow ."',
 '"Thierry Liebaut , director of development for a French media group , says if he had to pay top price for it in a restaurant , `` I would make t

In [ ]:
inputs = tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [ ]:
inputs

{'input_ids': tensor([[  101,  1000,  2074,  ...,  3700,  2011,   102],
        [  101,  1000,  2007,  ...,  2107, 12530,   102],
        [  101,  1000,  1036,  ...,  1005,  1005,   102],
        ...,
        [  101,  1000,  1054,  ...,  1998,  2008,   102],
        [  101,  1000,  1996,  ...,  1996,  2624,   102],
        [  101,   102,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ...,  True, False, False],
        [False, False, False,  ..., False, False, False]])

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

[[], [3, 17, 19], [6, 7, 13], [1], [1, 4, 5, 18, 19]]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
inputs.input_ids

tensor([[ 101, 1000, 2074,  ..., 3700, 2011,  102],
        [ 101, 1000, 2007,  ..., 2107,  103,  102],
        [ 101, 1000, 1036,  ..., 1005, 1005,  102],
        ...,
        [ 101, 1000, 1054,  ..., 1998, 2008,  102],
        [ 101, 1000, 1996,  ...,  103, 2624,  102],
        [ 101,  102,    0,  ...,    0,    0,    0]])

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#model.to(device)

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/73 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 73/73 [02:01<00:00,  1.67s/it, loss=0.00431]


##XLM

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')

In [ ]:
inputs = xlmroberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [ ]:
inputs

{'input_ids': tensor([[     0,     44, 127794,  ..., 181991,     53,      2],
        [     0,     44, 194583,  ..., 111719,      7,      2],
        [     0,     44, 164721,  ...,   5106,  17378,      2],
        ...,
        [     0,     44,   1052,  ...,  18153,      7,      2],
        [     0,     44,   3957,  ...,  43144,   3432,      2],
        [     0,      2,      1,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        [False, False, False,  ..., False, False,  True],
        ...,
        [False, False,  True,  ..., False,  True,  True],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False]])

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

[[10], [2, 7, 10], [4, 13, 20], [], [1, 7, 13, 14]]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
inputs.input_ids

tensor([[     0,     44, 127794,  ..., 181991,     53,      2],
        [     0,     44,    103,  ..., 111719,      7,      2],
        [     0,     44, 164721,  ...,   5106,  17378,    103],
        ...,
        [     0,     44,    103,  ...,  18153,    103,    103],
        [     0,     44,   3957,  ...,  43144,   3432,      2],
        [     0,      2,      1,  ...,      1,    103,      1]])

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#xlmroberta_model.to(device)

In [ ]:
from transformers import AdamW

# activate training mode
xlmroberta_model.train()
# initialize optimizer
optim = AdamW(xlmroberta_model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = xlmroberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/73 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 73/73 [05:16<00:00,  4.33s/it, loss=0.295]


In [ ]:
bert_model = model.eval()

In [ ]:
xlmroberta_model = xlmroberta_model.eval()

##BART

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [ ]:
inputs = bart_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [ ]:
inputs

{'input_ids': tensor([[    0,   113,  6785,  ...,    29,  4284,     2],
        [    0,   113,  3908,  ...,   215,  6723,     2],
        [    0,   113, 49519,  ...,  2431,  4273,     2],
        ...,
        [    0,   113,   500,  ...,     8,    14,     2],
        [    0,   113,   133,  ...,     5,   764,     2],
        [    0,     2,     1,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

tensor([[False, False, False,  ..., False, False,  True],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False],
        ...,
        [False, False, False,  ..., False, False,  True],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False]])

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

[[4, 5, 13, 20], [4, 13], [3, 9, 11, 13, 15, 19], [9], [1, 2, 5, 7]]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
inputs.input_ids

tensor([[    0,   113,  6785,  ...,    29,  4284,   103],
        [    0,   113,  3908,  ...,   215,  6723,     2],
        [    0,   113, 49519,  ...,  2431,   103,     2],
        ...,
        [    0,   113,   500,  ...,     8,    14,   103],
        [    0,   113,   133,  ...,     5,   764,     2],
        [    0,     2,     1,  ...,     1,   103,     1]])

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
from transformers import AdamW

# activate training mode
bart_model.train()
# initialize optimizer
optim = AdamW(bart_model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = bart_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/73 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 73/73 [07:00<00:00,  5.76s/it, loss=0.662]


In [ ]:
bart_model = bart_model.eval()

##RoBERTa

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [ ]:
inputs = roberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [ ]:
inputs

{'input_ids': tensor([[    0,   113,  6785,  ...,    29,  4284,     2],
        [    0,   113,  3908,  ...,   215,  6723,     2],
        [    0,   113, 49519,  ...,  2431,  4273,     2],
        ...,
        [    0,   113,   500,  ...,     8,    14,     2],
        [    0,   113,   133,  ...,     5,   764,     2],
        [    0,     2,     1,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

tensor([[False, False, False,  ...,  True, False,  True],
        [False,  True, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False]])

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

[[8, 13, 18, 20], [1, 7, 8, 16], [2, 5], [], [1]]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
inputs.input_ids

tensor([[   0,  113, 6785,  ...,  103, 4284,  103],
        [   0,  103, 3908,  ...,  215, 6723,    2],
        [   0,  113,  103,  ..., 2431, 4273,    2],
        ...,
        [   0,  113,  500,  ...,    8,   14,    2],
        [   0,  113,  133,  ...,    5,  764,    2],
        [   0,    2,    1,  ...,    1,  103,    1]])

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
from transformers import AdamW

# activate training mode
roberta_model.train()
# initialize optimizer
optim = AdamW(roberta_model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = roberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/73 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 73/73 [02:16<00:00,  1.88s/it, loss=0.16]


In [ ]:
roberta_model = roberta_model.eval()

##Masked Language Modeling Pipeline

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [ ]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

In [ ]:
top_k = 10

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

## after bart and roberta fine-tuning

In [ ]:
text_sentence = "The wildfire <mask> through the forest at an amazing speed."
get_all_predictions(text_sentence)

The wildfire <mask> through the forest at an amazing speed.


{'bart': 'tore\nwas\nripped\nraced\nroared',
 'bert': 'ripped\ntore\nmoved\nswept\nspread',
 'electra': 'runs\npasses\nflows\nsweeps\nswept',
 'roberta': 'raged\ntore\nmoved\nburned\nroared',
 'xlm': 'went\nrode\nsank\npassed\nran',
 'xlnet': 'through\nwildfire\nfire\nroaring\ncircles'}

##after bart and roberta fine-tuning

In [ ]:
text_sentence = "The window panes were <mask> as the wind blew through them"
get_all_predictions(text_sentence)

The window panes were <mask> as the wind blew through them


{'bart': 'knocked\nflying\nshaking\nratt\nfalling',
 'bert': 'shattered\ncracked\nblackened\nmissing\nrattled',
 'electra': 'clear\nvisible\ncovered\nclosed\nopen',
 'roberta': 'shaking\nwhite\ntrembling\nshattered\nopaque',
 'xlm': 'closed\nlit\nempty\ncovered\nbroken',
 'xlnet': 'as\njust\nwere\nsuch\nopen'}

## after bart and roberta fine-tuning

In [ ]:
text_sentence = "To heal and <mask> from death my heart"
get_all_predictions(text_sentence)

To heal and <mask> from death my heart


{'bart': 'escape\nsave\nrecover\nrelease\nprotect',
 'bert': 'save\nprotect\nrecover\nkeep\nfree',
 'electra': 'heal\nrecover\ndie\nlive\nsave',
 'roberta': 'protect\nrelease\nescape\nrecover\nkeep',
 'xlm': 'save\nprotect\nsalve\nescape\nfree',
 'xlnet': 'and\nfrom\ncome\nright\naway'}

In [ ]:
text_sentence ="The tax cut will <mask> the economy"
get_all_predictions(text_sentence)

The tax cut will <mask> the economy


{'bart': 'boost\nrevive\nstimulate\nhelp\nfix',
 'bert': 'help\nhurt\ncool\nboost\nkill',
 'electra': 'hurt\nhelp\nboost\nbenefit\nimprove',
 'roberta': 'boost\nhelp\nstimulate\nstrengthen\nfix',
 'xlm': 'strike\nkill\nboost\nfuel\nbenefit',
 'xlnet': 'all\nwill\nbe\nnot\nthe'}

In [ ]:
text_sentence = "Black desert <mask> in iron silences"
get_all_predictions(text_sentence)

Black desert <mask> in iron silences


{'bart': 'fires\nburns\nsm\nwinds\nr',
 'bert': 'nights\ners\nwinds\ntemperatures\nwandering',
 'electra': 'is\nappears\nflies\nes\nfalls',
 'roberta': 'hides\nsurvives\nfloats\nlies\nwrapped',
 'xlm': 's\nfilled\ncovered\nliving\nswimming',
 'xlnet': 'and\nfor\nin\nis\non'}

##Wikipedia

In [ ]:
text_sentence = "0.000123, which  <mask> to a distance of 705 Mly, or 216 Mpc.000webhost is a free web hosting service"
get_all_predictions(text_sentence)

0.000123, which  <mask> to a distance of 705 Mly, or 216 Mpc.000webhost is a free web hosting service


{'bart': 'is\nconnects\nflies\nextends\ncan',
 'bert': 'extends\ncorresponds\ntranslates\nruns\ntravels',
 'electra': 'extends\nruns\nvaries\napplies\noperates',
 'roberta': 'corresponds\namounts\nequals\nextends\nstretches',
 'xlm': 'runs\ngoes\nis\npoints\nfalls',
 'xlnet': 'is\ncomes\ncame\nwas\nwent'}

In [ ]:
text_sentence = "operated by Hostinger.0010x0010 is a Dutch- <mask> audiovisual artist"
get_all_predictions(text_sentence)

operated by Hostinger.0010x0010 is a Dutch- <mask> audiovisual artist


{'bart': 'based\nGerman\nIsraeli\nGerman\nFrench',
 'bert': 'born\nbased\namerican\ncanadian\nspeaking',
 'electra': 'based\nborn\namerican\nlanguage\nowned',
 'roberta': 'born\nbased\nAmerican\ntrained\nJewish',
 'xlm': 'American\nborn\nEnglish\nEuropean\nGerman',
 'xlnet': 'on\nmore\nthe\na\nnational'}

In [ ]:
text_sentence = "an alcohol abuse prevention program  <mask> in 2004 at Francis E. Warren Air Force Base"
get_all_predictions(text_sentence)

an alcohol abuse prevention program  <mask> in 2004 at Francis E. Warren Air Force Base


{'bart': 'was\nbegan\nstarted\nlaunched\nfirst',
 'bert': 'began\nestablished\nstarted\nopened\nlaunched',
 'electra': 'began\nstarted\nlaunched\nopened\nestablished',
 'roberta': 'began\nstarted\nopened\nlaunched\nbeginning',
 'xlm': 'started\nbegan\nopened\ndeveloped\nclosed',
 'xlnet': 'in\nprogram\nbegan\nbeginning\nstarted'}

In [ ]:
text_sentence = "<mask> on research by the National Institute on Alcohol Abuse and Alcoholism regarding binge drinking in college students.0.01"
get_all_predictions(text_sentence)

<mask> on research by the National Institute on Alcohol Abuse and Alcoholism regarding binge drinking in college students.0.01


{'bart': '<s>\n0\nThe\n1\n</s>',
 'bert': 'based\nreport\nbase\nreliance\nbasis',
 'electra': 'based\ninformation\nreport\nfocusing\ndata',
 'roberta': 'Based\nDraw\nFocus\nbased\nInformation',
 'xlm': '</s>\nComment\nΆρθρο\nReport\nREPORT',
 'xlnet': 'based\ninformation\nBased\ndata\nreport'}

In [ ]:
text_sentence = "is the debut studio album of H3llb3nt,  <mask> on February 20, 1996 by Fifth Colvmn Records.001 of 3 February 1997"
get_all_predictions(text_sentence)

is the debut studio album of H3llb3nt,  <mask> on February 20, 1996 by Fifth Colvmn Records.001 of 3 February 1997


{'bart': 'released\nissued\nrecorded\noriginally\nfirst',
 'bert': 'released\nissued\nrecorded\nreleasing\nannounced',
 'electra': 'released\nreissued\nrecorded\npublished\nrelease',
 'roberta': 'released\nissued\ndropped\ndistributed\nrecorded',
 'xlm': 'released\nreported\ndue\npublished\nrelease',
 'xlnet': 'released\nissued\npublished\nReleased\ndistributed'}

In [ ]:
text_sentence = " which was  <mask> between the Government of the Republic of Rwanda, and FAPADER.003230 is a South Korean food manufacturer."
get_all_predictions(text_sentence)

 which was  <mask> between the Government of the Republic of Rwanda, and FAPADER.003230 is a South Korean food manufacturer.


{'bart': 'formed\na\nestablished\nfounded\ncreated',
 'bert': 'split\nfought\nnegotiated\nsettled\ndivided',
 'electra': 'negotiated\njointly\nshared\nagreed\nsplit',
 'roberta': 'negotiated\nforged\narranged\nformed\nagreed',
 'xlm': 'agreed\nsigned\njoint\nbuilt\ndeveloped',
 'xlnet': 'between\nthat\nmidway\nwas\nin'}

In [ ]:
text_sentence = "Gas molecules in soil are in continuous thermal motion  <mask> to the kinetic theory of gasses, there is also collision between molecules - a random walk.0.04"
get_all_predictions(text_sentence)

Gas molecules in soil are in continuous thermal motion  <mask> to the kinetic theory of gasses, there is also collision between molecules - a random walk.0.04


{'bart': '."\nbecause\nbut\nand\nthrough',
 'bert': 'according\ndue\nsimilar\nthanks\ncontrary',
 'electra': 'according\nsimilar\ndue\ncompared\ncontrary',
 'roberta': 'According\nTrue\naccording\nDue\nThanks',
 'xlm': 'due\nbecause\nAccording\n--\naccording',
 'xlnet': 'according\nAccording\nAccording\ndue\naccording'}

In [ ]:
text_sentence = "is a first-person shooter video game <mask> the character of British secret agent James Bond."
get_all_predictions(text_sentence)

is a first-person shooter video game <mask> the character of British secret agent James Bond.


{'bart': 'featuring\nstarring\nbased\nthat\ndepicting',
 'bert': 'featuring\nstarring\nabout\ndepicting\nexploring',
 'electra': 'in\nfeaturing\nstarring\nwith\nabout',
 'roberta': 'exploring\nfeaturing\nstarring\nwith\nusing',
 'xlm': 'featuring\nabout\nwith\nin\nplaying',
 'xlnet': 'about\nwith\nfor\nfeaturing\non'}

In [ ]:
text_sentence = "007 Legends is the fourth and final James Bond game title  <mask> by Activision, the last game"
get_all_predictions(text_sentence)

007 Legends is the fourth and final James Bond game title  <mask> by Activision, the last game


{'bart': 'published\ndeveloped\nreleased\nproduced\nmade',
 'bert': 'produced\nreleased\npublished\ndeveloped\nmade',
 'electra': 'owned\nreleased\nissued\nacquired\nsigned',
 'roberta': 'developed\npublished\nreleased\nproduced\ndistributed',
 'xlm': 'developed\nproduced\nreleased\nmade\nplayed',
 'xlnet': 'title\nreleased\npublished\nplayed\nmade'}

In [ ]:
text_sentence = "<mask> on the James Bond license.00"
get_all_predictions(text_sentence)

<mask> on the James Bond license.00


{'bart': '<s>\nThe\n</s>\nIt\nHe',
 'bert': 'based\nvariations\nnot\nclick\ndepends',
 'electra': '00\n01\n50\n45\n30',
 'roberta': 'Notes\nNote\nWork\nInformation\nConnect',
 'xlm': '</s>\nრეკლამა\nപിഴ\nՄանրամասն\nຮຽນພາສາອັງກິດ',
 'xlnet': 'on\non\nen\n00\n02'}

In [ ]:
text_sentence = "HappyPeeps.0.0 is a live album by Melt-Banana that  <mask> out on November 3, 2009.Melt"
get_all_predictions(text_sentence)

HappyPeeps.0.0 is a live album by Melt-Banana that  <mask> out on November 3, 2009.Melt


{'bart': 'came\nwas\ncomes\nfirst\nis',
 'bert': 'came\nwent\nsold\nrolled\ntopped',
 'electra': 'came\ncomes\ncome\nwent\nopened',
 'roberta': 'came\nwas\ncomes\nwent\nis',
 'xlm': 'went\ncame\ncomes\nis\ngoes',
 'xlnet': 'came\ncomes\nwas\nis\nwent'}

In [ ]:
text_sentence = "Melt-Banana Lite, which  <mask> that they  <mask> their music with a different arrangement"
get_all_predictions(text_sentence)

Melt-Banana Lite, which  <mask> that they  <mask> their music with a different arrangement


{'bart': 'announced\nclaims\nsays\nsaid\nclaimed',
 'bert': 'claims\ninsists\nsays\nsaid\nstated',
 'electra': 'means\nsays\nshows\nensures\nsuggests',
 'roberta': 'means\nindicates\nnotes\nmeant\nadded',
 'xlm': 'means\nis\nfound\nshows\nstates',
 'xlnet': 'which\nthat\nmeans\nis\nthis'}

In [ ]:
text_sentence = ".0.0% is  <mask> to be the start of the retracement, while 100.0% is a complete reversal to the original part of the move"
get_all_predictions(text_sentence)

.0.0% is  <mask> to be the start of the retracement, while 100.0% is a complete reversal to the original part of the move


{'bart': 'expected\ngenerally\nbelieved\nlikely\nconsidered',
 'bert': 'said\nlikely\nsupposed\nintended\nexpected',
 'electra': 'supposed\nexpected\nsaid\nlikely\ngoing',
 'roberta': 'expected\nbelieved\nconsidered\nlikely\nthought',
 'xlm': 'expected\nsaid\nlikely\nthought\nreported',
 'xlnet': 'just\nnot\nsupposed\nintended\nset'}

In [ ]:
text_sentence = "00 pm in daily  <mask> on rerun programs to simultaneously networks.00 -"
get_all_predictions(text_sentence)

00 pm in daily  <mask> on rerun programs to simultaneously networks.00 -


{'bart': 'programming\nbroadcasts\ntime\nnews\neditions',
 'bert': 'and\nor\nnews\nintervals\nupdates',
 'electra': 'updates\ntime\ntimes\ndays\nhours',
 'roberta': 'rotation\nblock\ntrading\nbroadcasts\ncredits',
 'xlm': 'reports\nspending\nban\nwork\nwar',
 'xlnet': 'daily\non\nprogram\nprograms\ndaytime'}

In [ ]:
text_sentence = "The rebels  <mask> to  <mask> objects with a catapult."
get_all_predictions(text_sentence)

The rebels  <mask> to  <mask> objects with a catapult.


{'bart': 'used\ntry\nuse\nplan\nare',
 'bert': 'began\ntried\nattempted\nproceeded\ncontinued',
 'electra': 'began\ntried\nproceeded\nattempted\nhad',
 'roberta': 'tried\nused\nbegan\ntend\ntry',
 'xlm': 'tried\nplan\nattempt\nplans\ntry',
 'xlnet': 'to\nagree\nare\non\ngo'}

In [ ]:
text_sentence = "01-04-043 as  <mask> in the catalog.010 being the areacode for the city of Rotterdam"
get_all_predictions(text_sentence)

01-04-043 as  <mask> in the catalog.010 being the areacode for the city of Rotterdam


{'bart': 'shown\ndescribed\nthe\ndepicted\nlisted',
 'bert': 'shown\nlisted\nseen\ndefined\nfound',
 'electra': 'shown\nincluded\nlisted\nspecified\ndescribed',
 'roberta': 'described\nindicated\nshown\ndefined\nnoted',
 'xlm': 'shown\ndescribed\nmentioned\nreported\nlisted',
 'xlnet': 'as\nused\nshown\ndescribed\nstated'}

In [ ]:
text_sentence = "01.10  <mask> to January 2010"
get_all_predictions(text_sentence)

01.10  <mask> to January 2010


{'bart': 'February\nfrom\nJune\nApril\nJanuary',
 'bert': 'up\nthrough\nyears\nyear\n...',
 'electra': 'compared\ndates\nyear\nmarch\nprior',
 'roberta': 'September\nJun\nonwards\nprior\nJune',
 'xlm': 'ოქტომბერი\nlähtien\nalkaen\nഫെബ്രുവരി\nივლისი',
 'xlnet': 'to\nand\nfrom\nin\ns'}

In [ ]:
text_sentence = "Revolution in Just  <mask>"
get_all_predictions(text_sentence)

Revolution in Just  <mask>


{'bart': 'Society\nIdeas\nPlain\nEngland\nAmerica',
 'bert': 'days\nbeginning\nmonths\nwords\nweeks',
 'electra': 'just\none\nright\n1\n2',
 'roberta': 'icum\nine\nicia\nus\nCause',
 'xlm': 'ກ່ຽວກັບພວກເຮົາ\nລົດ\nຕິດຕາມພວກເຮົາ\ntýžden\nനിമിഷം',
 'xlnet': 'Just\nSo\nAnd\nU\nBut'}

In [ ]:
text_sentence = "01:59PM is the first studio album by South Korean boy band, 2PM.01708 is the national dialing code for Romford in the United Kingdom.0181 is a compilation album by Four Tet,  <mask> by Text Records"
get_all_predictions(text_sentence)

01:59PM is the first studio album by South Korean boy band, 2PM.01708 is the national dialing code for Romford in the United Kingdom.0181 is a compilation album by Four Tet,  <mask> by Text Records


{'bart': 'released\ndistributed\nissued\npublished\noriginally',
 'bert': 'released\ndistributed\nissued\nproduced\nreissued',
 'electra': 'produced\nreleased\npublished\nand\ncompiled',
 'roberta': 'distributed\nreleased\npresented\nproduced\nissued',
 'xlm': 'released\nproduced\nplayed\npublished\nsigned',
 'xlnet': 'released\npublished\nproduced\nissued\navailable'}

In [ ]:
text_sentence = "<mask> that the California Department of Corrections and Rehabilitation's (CDCR)"
get_all_predictions(text_sentence)

<mask> that the California Department of Corrections and Rehabilitation's (CDCR)


{'bart': '<s>\nIt\nThe\n</s>\nHe',
 'bert': 'arguing\nadding\nadded\nand\nafter',
 'electra': 'note\nnotice\nreports\nreport\nnotes',
 'roberta': 'Note\nAnd\nAdded\nadded\nAdding',
 'xlm': 'Note\nAfter\n--\nSaid\nExcept',
 'xlnet': 'that\nis\ns\nof\nwas'}

In [ ]:
text_sentence = "medical services are inadequate and  <mask>"
get_all_predictions(text_sentence)

medical services are inadequate and  <mask>


{'bart': 'under\ncostly\nexpensive\nunavailable\ninaccessible',
 'bert': 'ineffective\nunreliable\nfailing\ninadequate\ninsufficient',
 'electra': 'inadequate\nexpensive\nineffective\nunnecessary\ncostly',
 'roberta': 'unavailable\nunreliable\nexpensive\ninaccessible\ncostly',
 'xlm': 'expensive\naffordable\ninconvenient\ninvalid\ndangerous',
 'xlnet': 'are\nand\nthat\na\nis'}

In [ ]:
text_sentence = "<mask> in 2008.0.1% of all web pages  <mask> Windows-1254 in February 2016.01 "
get_all_predictions(text_sentence)

<mask> in 2008.0.1% of all web pages  <mask> Windows-1254 in February 2016.01 


{'bart': '<s>\n0\n1\n0\nThe',
 'bert': 'growth\n10\nrate\nbeginning\nfound',
 'electra': '1\n0\n01\n2\nposted',
 'roberta': 'Starting\nWhereas\nBeginning\nAnd\nBut',
 'xlm': '</s>\nശേഖരിച്ചത്\nPosted\nสงวนลิขสิทธิ์\nPublished',
 'xlnet': '54\n24\n56\n4\n04'}

In [ ]:
text_sentence = "<mask> in 2006 by Bom Kim and Daniel Loss, later  <mask> by Atlantic Media (David Bradley); it  <mask> graduates of Harvard University.021"
get_all_predictions(text_sentence)

<mask> in 2006 by Bom Kim and Daniel Loss, later  <mask> by Atlantic Media (David Bradley); it  <mask> graduates of Harvard University.021


{'bart': '<s>\nThe\nIt\n</s>\nIn',
 'bert': 'founded\nestablished\ncreated\nstarted\nlaunched',
 'electra': 'published\nfounded\nlaunched\nreleased\ncreated',
 'roberta': 'Created\nPublished\nfounded\nestablished\nWritten',
 'xlm': 'Published\nWritten\nPosted\nFound\nLed',
 'xlnet': 'Incorporated\nproduced\nDistributed\npopulated\ncommissioned'}

In [ ]:
text_sentence = "<mask> from pGreenII 0229"
get_all_predictions(text_sentence)

<mask> from pGreenII 0229


{'bart': '<s>\nThe\n</s>\nHe\nIn',
 'bert': 'data\nsourced\nmaterial\nretrieved\nextracted',
 'electra': 'read\navailable\ndata\nvideo\noutput',
 'roberta': 'Information\nData\nImage\nSent\nText',
 'xlm': 'ဖိုးသူတော်\nဓာတ်ပုံများ\nກ່ຽວກັບພວກເຮົາ\nΑναρτήθηκε\n</s>',
 'xlnet': 'from\non\na\nor\nfar'}

In [ ]:
text_sentence = "9-time All-American while  <mask> for 4 years.02 9D 74 E3 5B D8 41"
get_all_predictions(text_sentence)

9-time All-American while  <mask> for 4 years.02 9D 74 E3 5B D8 41


{'bart': 'playing\nleading\ncoaching\ncapt\nactive',
 'bert': 'playing\nthere\nserving\ncompeting\nwrestling',
 'electra': 'serving\nplaying\nworking\nliving\nrunning',
 'roberta': 'playing\nstarting\ncovering\nstarring\nthere',
 'xlm': 'sitting\nworking\nliving\npregnant\nplaying',
 'xlnet': 'for\nrunning\nplaying\nonly\njust'}

In [ ]:
text_sentence = "56 C5 63 56 88 C0 (commonly  <mask> to as 09 F9).02-cv-0299 is a habeas corpus petition"
get_all_predictions(text_sentence)

56 C5 63 56 88 C0 (commonly  <mask> to as 09 F9).02-cv-0299 is a habeas corpus petition


{'bart': 'referred\nknown\nrefers\nrefer\natt',
 'bert': 'referred\nrefer\nrefers\nabbreviated\nobjected',
 'electra': 'referred\nrefer\nknown\nspoken\ntranslated',
 'roberta': 'referred\nshortened\nReferred\nalluded\nattended',
 'xlm': 'reported\ncompared\nassign\nequal\nheld',
 'xlnet': 'referred\nrefer\nreferring\nrefers\nknown'}

In [ ]:
text_sentence = "<mask> album, after Antisleep Vol.0.2% of the population"
get_all_predictions(text_sentence)

<mask> album, after Antisleep Vol.0.2% of the population


{'bart': '<s>\n0\n1\nThe\n2',
 'bert': 'the\nthis\nsecond\nanother\nthird',
 'electra': 'the\nthis\nanother\nan\nhis',
 'roberta': 'The\nsecond\nThat\nSecond\nThis',
 'xlm': 'Previous\n</s>\nLast\nThis\nThe',
 'xlnet': 'the\nsoon\nyear\nalbum\nmajor'}

In [ ]:
text_sentence = "0.2% of the population  <mask> in the service industry and 0.1% works in the fishing"
get_all_predictions(text_sentence)

0.2% of the population  <mask> in the service industry and 0.1% works in the fishing


{'bart': 'works\nis\nworked\nwork\nwas',
 'bert': 'works\nis\nwork\nworked\nemployed',
 'electra': 'works\nwork\nis\noperates\nplays',
 'roberta': 'works\nis\nwork\nare\njobs',
 'xlm': 'works\nwork\nis\nlives\nworkers',
 'xlnet': 'works\nwork\nis\nlives\nare'}

In [ ]:
text_sentence = "industry.0.2%.The school was home to Detroit Pistons player Richard Rip Hamilton.03-04  <mask> on September 25, 2003"
get_all_predictions(text_sentence)

industry.0.2%.The school was home to Detroit Pistons player Richard Rip Hamilton.03-04  <mask> on September 25, 2003


{'bart': 'class\ntournament\nTournament\nfinals\nyear',
 'bert': 'closed\nheld\nmerger\ngraduation\nest',
 'electra': '–\nvictory\nearly',
 'roberta': 'opened\nclosed\ndied\nbegan\ngraduating',
 'xlm': '</s>\nshooting\nlives\nPosted\ndating',
 'xlnet': 'born\non\nreleased\nmade\n02'}

In [ ]:
text_sentence = "is an English-language, bi-annual contemporary culture magazine that  <mask> art, fashion, and politics.03 and the Blackstar score"
get_all_predictions(text_sentence)

is an English-language, bi-annual contemporary culture magazine that  <mask> art, fashion, and politics.03 and the Blackstar score


{'bart': 'covers\nexamines\nexplores\nfeatures\nfocuses',
 'bert': 'covers\ndiscusses\nexamines\nexplores\nreviews',
 'electra': 'covers\npublishes\nfeatures\nexplores\ndiscusses',
 'roberta': 'covers\nexamines\nexplores\ncelebrates\ninvestigates',
 'xlm': 'features\nlinks\ncover\nsports\ncovered',
 'xlnet': 'covers\npublishes\nincludes\nfeatures\ncover'}

In [ ]:
text_sentence = "03 is Isis's third live release.03 is the fourth album by Blue Stahli, and his third instrumental- <mask> album, after Antisleep Vol.0.3 mile south is"
get_all_predictions(text_sentence)

03 is Isis's third live release.03 is the fourth album by Blue Stahli, and his third instrumental- <mask> album, after Antisleep Vol.0.3 mile south is


{'bart': 'based\ndriven\nmusic\noriented\nd',
 'bert': 'oriented\nbased\nonly\nrock\npop',
 'electra': 'solo\ninstrumental\nrelease\nplay\ndriven',
 'roberta': 'rock\noriented\nonly\ndriven\npop',
 'xlm': 'style\nsong\nlong\ncore\n</s>',
 'xlnet': 'based\nonly\noriented\nlength\nalbum'}

In [ ]:
text_sentence = "Thrumster Parish Church, part of the Church of Scotland charge of Pulteneytown and Thrumster.0.3% of ReadyReturn filings  <mask> errors, versus 3.1% of non-ReadyReturn filings.0.3 TBq is equal to 1/40th of the natural radiation in one km3 of seawater and 1/50000000000th of the total natural radioactivity in the oceans.04092001"
get_all_predictions(text_sentence)

Thrumster Parish Church, part of the Church of Scotland charge of Pulteneytown and Thrumster.0.3% of ReadyReturn filings  <mask> errors, versus 3.1% of non-ReadyReturn filings.0.3 TBq is equal to 1/40th of the natural radiation in one km3 of seawater and 1/50000000000th of the total natural radioactivity in the oceans.04092001


{'bart': 'had\ncontain\nhave\ncontained\nsuffer',
 'bert': 'contain\nreport\nshow\nare\ndisplay',
 'electra': 'are\nhave\ncontain\nwere\nreport',
 'roberta': 'contain\ncontained\nreport\nshow\nadmit',
 'xlm': 'reported\nhad\nfound\nreport\ncause',
 'xlnet': 'contain\nwere\ncontained\nare\nhad'}

In [ ]:
text_sentence = "is the fourth studio album by South Korean trio, Urban Zakapa,  <mask> of 9 track "
get_all_predictions(text_sentence)

is the fourth studio album by South Korean trio, Urban Zakapa,  <mask> of 9 track 


{'bart': 'consisting\na\ncomposed\nand\nfeaturing',
 'bert': 'consisting\nconsists\ncomposed\nconsist\nconsisted',
 'electra': 'consisting\ncomposed\nconsists\nout\nset',
 'roberta': 'consisting\ncomposed\ncomprised\ncomprising\nconsists',
 'xlm': 'set\nseries\nconsist\nas\nmade',
 'xlnet': 'and\nout\nconsisting\nwhich\nas'}

In [ ]:
text_sentence = "<mask> on smaller frigates, so an alternative had to be sought.055 Brigade (or 55th Arab Brigade)"
get_all_predictions(text_sentence)

<mask> on smaller frigates, so an alternative had to be sought.055 Brigade (or 55th Arab Brigade)


{'bart': '<s>\n055\nThe\n05\nIt',
 'bert': 'particularly\nespecially\nand\nbut\nexcept',
 'electra': 'especially\nnot\nbased\nbuilt\nparticularly',
 'roberta': 'rely\nrang\nworking\nlying\nlied',
 'xlm': "``\n</s>\n''\n\x95\n--",
 'xlnet': 'operate\nused\nwork\nonly\nsailing'}

In [ ]:
text_sentence = "<mask> into the Taliban army between 1995 and 2001.0.5% calcium chlorideH6"
get_all_predictions(text_sentence)

<mask> into the Taliban army between 1995 and 2001.0.5% calcium chlorideH6


{'bart': '<s>\n0\n1\nThe\n0',
 'bert': 'inducted\nintegrated\npumped\ndrafted\nincorporated',
 'electra': 'released\ndischarged\nrecruited\nincorporated\ndrafted',
 'roberta': 'Sold\nlied\nIntroduced\nBorn\nreleased',
 'xlm': 'Breaking\n\x97\nTrip\nDrop\nentry',
 'xlnet': 'into\nentry\nentered\nin\nre'}

In [ ]:
text_sentence = "<mask> in a number of military applications, notably underwater munitions (e.g. naval mines, depth charges and torpedoes)"
get_all_predictions(text_sentence)

<mask> in a number of military applications, notably underwater munitions (e.g. naval mines, depth charges and torpedoes)


{'bart': '<s>\nThe\n</s>\nIn\nIt',
 'bert': 'used\napplications\nand\nengaged\nemployed',
 'electra': 'used\nuseful\ninvolved\nspecialized\nengaging',
 'roberta': 'Used\nuse\nused\nwork\nwork',
 'xlm': '``\n\x95\n•\n</s>\n--',
 'xlnet': 'used\nand\nas\nuse\nthem'}

In [ ]:
text_sentence = "where it has generally  <mask> torpex, being less shock sensitive and  "
get_all_predictions(text_sentence)

where it has generally  <mask> torpex, being less shock sensitive and  


{'bart': 'outper\nreplaced\nbeen\nout\nattacked',
 'bert': 'less\nmore\navoided\nabsorbed\nreplaced',
 'electra': 'been\nless\nmore\nthe\na',
 'roberta': 'absorbed\navoided\nimproved\nshed\nescaped',
 'xlm': 'been\ntried\nbecome\nkilled\ndesigned',
 'xlnet': 'been\ngenerally\ntypically\nmost\nusually'}

In [ ]:
text_sentence = "<mask> more stable storage characteristics.0.5 cl is approximately 1 U.S."
get_all_predictions(text_sentence)

<mask> more stable storage characteristics.0.5 cl is approximately 1 U.S.


{'bart': '<s>\n0\n5\n1\nThe',
 'bert': 'demonstrating\nfor\nwith\nshows\ndemonstrated',
 'electra': 'much\nno\nfor\nwith\nthe',
 'roberta': 'The\nIts\nHis\nMuch\nThe',
 'xlm': '``\n•\n</s>\n\x95\n·',
 'xlnet': 'more\nand\nwith\nas\nin'}

In [ ]:
text_sentence = "Ross.061502 is a live concert DVD of Botch's final show at The Showbox in Seattle.0.6.17 of Skencil was  <mask> in 2005, and a preview 1.0 alpha was  "
get_all_predictions(text_sentence)

Ross.061502 is a live concert DVD of Botch's final show at The Showbox in Seattle.0.6.17 of Skencil was  <mask> in 2005, and a preview 1.0 alpha was  


{'bart': 'released\nrecorded\nfirst\nproduced\noriginally',
 'bert': 'released\nannounced\nadded\ndiscontinued\nintroduced',
 'electra': 'released\nrecorded\nfilmed\nshown\nproduced',
 'roberta': 'released\nissued\npublished\ndistributed\nrecorded',
 'xlm': 'released\nshot\nout\nmade\nsold',
 'xlnet': 'released\nmade\nannounced\nproduced\npublished'}

In [ ]:
text_sentence = "in 2010.06 is a New Zealand compilation album  <mask>"
get_all_predictions(text_sentence)

in 2010.06 is a New Zealand compilation album  <mask>


{'bart': 'by\nfeaturing\nfrom\nreleased\nof',
 'bert': 'released\nrelease\nalbum\nep\npt',
 'electra': 'released\nalbum\nrelease\nrecord\ncompilation',
 'roberta': 'called\nentitled\non\ntitled\nreleased',
 'xlm': "</s>\n''\n``\ntitle\npromo",
 'xlnet': 'album\nand\ncollection\nthat\nof'}

In [ ]:
text_sentence = "service is  <mask> on the principle of a flexible virtual telephone number.07548 and pin code is 473443.Ganesh Shankar Vidyarthi Indian independence movement activist and journalist  "
get_all_predictions(text_sentence)

service is  <mask> on the principle of a flexible virtual telephone number.07548 and pin code is 473443.Ganesh Shankar Vidyarthi Indian independence movement activist and journalist  


{'bart': 'based\nrun\nbuilt\noffered\ndesigned',
 'bert': 'based\noperated\nrun\noffered\nconducted',
 'electra': 'based\nbuilt\nstructured\nprovided\noperated',
 'roberta': 'based\noperated\nfounded\nrun\noperating',
 'xlm': 'based\nbuilt\ndesigned\ndependent\nrun',
 'xlnet': 'based\nbuilt\noffered\navailable\noperating'}

In [ ]:
text_sentence = "Ganesh Shankar Vidyarthi Indian independence movement activist and journalist  <mask> here for sometime.07 Detva"
get_all_predictions(text_sentence)

Ganesh Shankar Vidyarthi Indian independence movement activist and journalist  <mask> here for sometime.07 Detva


{'bart': 'has\nwas\nis\nhad\nremained',
 'bert': 'lived\nstayed\nresided\nworked\nbased',
 'electra': 'is\nwas\nstays\nremains\nstayed',
 'roberta': 'was\nis\ndied\nlives\nlived',
 'xlm': 'lived\nwas\nworked\nserved\nmoved',
 'xlnet': 'here\nright\nthere\naround\non'}

In [ ]:
text_sentence = "-8-4 originally  <mask> on ABC on October 1, 2013"
get_all_predictions(text_sentence)

-8-4 originally  <mask> on ABC on October 1, 2013


{'bart': 'aired\npremiered\ndebuted\nran\nappeared',
 'bert': 'aired\nbroadcast\nair\npremiered\nappeared',
 'electra': 'aired\nappeared\npremiered\nannounced\nbroadcast',
 'roberta': 'aired\npremiered\nran\ndebuted\nbroadcast',
 'xlm': 'appeared\nreported\nbroadcast\nran\nplaced',
 'xlnet': 'appeared\nappears\nreported\non\naired'}

In [ ]:
text_sentence = "North Rhine-Westphalia.08-cv-34, was a decision in the Northern District of Florida which  <mask> a student's First Amendment right to   "
get_all_predictions(text_sentence)

North Rhine-Westphalia.08-cv-34, was a decision in the Northern District of Florida which  <mask> a student's First Amendment right to   


{'bart': 'struck\nattacked\nupheld\naffirmed\nblocked',
 'bert': 'recognized\naffirmed\nupheld\nviolated\ndenied',
 'electra': 'granted\nupheld\nincluded\nviolated\ngave',
 'roberta': 'affirmed\nprotected\nupheld\nprotects\nrecognized',
 'xlm': 'ended\nclosed\nheld\nremoved\nbar',
 'xlnet': 'upheld\naffected\ngave\nestablished\ndeprived'}

In [ ]:
text_sentence = "Responding to pressure from the African-American community, the character Lil' Eightball was  <mask>"
get_all_predictions(text_sentence)

Responding to pressure from the African-American community, the character Lil' Eightball was  <mask>


{'bart': 'killed\ndropped\nremoved\ncast\nblack',
 'bert': 'created\nredesigned\nintroduced\ndropped\nreintroduced',
 'electra': 'born\nreleased\nintroduced\neliminated\nkilled',
 'roberta': 'resurrected\nkilled\nrenamed\nrewritten\ncreated',
 'xlm': 'born\ncreated\ncalled\ngiven\nplaying',
 'xlnet': 'a\n“\nnamed\nwas\nnot'}

In [ ]:
text_sentence = "<mask> in a non-composited mode.0.9 is the fourth album by French rapper Booba"
get_all_predictions(text_sentence)

<mask> in a non-composited mode.0.9 is the fourth album by French rapper Booba


{'bart': '<s>\n0\n1\n0\n</s>',
 'bert': 'live\nmusic\nrecorded\nreissued\nart',
 'electra': 'released\nperformed\nrecorded\nwritten\nset',
 'roberta': 'released\nReleased\nWritten\nSent\nNow',
 'xlm': '</s>\nלצפיה\n·\n``\nΕγγραφή',
 'xlnet': 'in\nmode\nstate\nit\nsystem'}

In [ ]:
text_sentence = "Sri Lankans were eligible to  <mask> in the election.09 Towers is a complex of six towers"
get_all_predictions(text_sentence)

Sri Lankans were eligible to  <mask> in the election.09 Towers is a complex of six towers


{'bart': 'vote\nparticipate\ncast\nrun\nchoose',
 'bert': 'vote\nrun\nstand\ncontest\nparticipate',
 'electra': 'vote\nparticipate\nrun\nwin\ncompete',
 'roberta': 'vote\nparticipate\nstand\nrun\ncontest',
 'xlm': 'vote\nremain\nrun\nstemme\nsit',
 'xlnet': 'vote\nparticipate\nin\nbe\nto'}

In [ ]:
text_sentence = "09 Towers is a complex of six towers  <mask> across a  built-up area of 450,000 square meters."
get_all_predictions(text_sentence)

09 Towers is a complex of six towers  <mask> across a  built-up area of 450,000 square meters.


{'bart': 'spread\nspanning\nbuilt\nscattered\nspreading',
 'bert': 'spread\nscattered\nstretching\nstretched\nspreading',
 'electra': 'built\nrunning\nstretching\nlocated\nextending',
 'roberta': 'spread\nscattered\nspaced\nstretched\nspanning',
 'xlm': 'spread\nlocated\n</s>\nrunning\nbuilt',
 'xlnet': 'spread\ntowers\nspanning\nset\nbased'}

In [ ]:
text_sentence = "squad.0 A.D. is a free and open-source, real-time strategy game under development by Wildfire Games.0DFx (also  <mask> as Zero Defects or Zero Defex) is an American hardcore punk band.0% financing (zero percent)"
get_all_predictions(text_sentence)

squad.0 A.D. is a free and open-source, real-time strategy game under development by Wildfire Games.0DFx (also  <mask> as Zero Defects or Zero Defex) is an American hardcore punk band.0% financing (zero percent)


{'bart': 'known\nreferred\nbilled\ncredited\nknow',
 'bert': 'known\nreleased\nstylized\nmarketed\nspelled',
 'electra': 'known\nclassified\nmarketed\nabbreviated\nlicensed',
 'roberta': 'known\nknow\nbranded\ncredited\nspelled',
 'xlm': 'known\ncalled\ndescribed\nregistered\nidentified',
 'xlnet': 'known\nknow\navailable\nreleased\ncalled'}

In [ ]:
text_sentence = "is an American hardcore punk band.0% financing (zero percent), alternatively  <mask>"
get_all_predictions(text_sentence)

is an American hardcore punk band.0% financing (zero percent), alternatively  <mask>


{'bart': 'known\ndescribed\ncalled\navailable\nreferred',
 'bert': 'written\nspelled\ntitled\ncalled\nstyled',
 'electra': 'not\neither\nfunded\nsold\nincluded',
 'roberta': 'TBA\nsourced\nfinanced\nlend\narranged',
 'xlm': '</s>\nhas\na\ngroup',
 'xlnet': 'to\nthat\ncalled\nfrom'}

In [ ]:
text_sentence = "marketing tactic for  <mask> buyers of consumer goods, automobiles, real estate, or credit cards in different parts of the world.0 has been  <"
get_all_predictions(text_sentence)

marketing tactic for  <mask> buyers of consumer goods, automobiles, real estate, or credit cards in different parts of the world.0 has been  <


{'bart': 'affluent\nattracting\nforeign\npotential\nconsumers',
 'bert': 'young\nprospective\namerican\ntarget\nwilling',
 'electra': 'the\nall\nmany\ndifferent\nboth',
 'roberta': 'small\nyoung\ntargeting\naffluent\nconvincing',
 'xlm': 'retail\n</s>\nforeign\nthe\nnew',
 'xlnet': 'the\nvarious\ncommercial\npotential\nprospective'}

In [ ]:
text_sentence = "that  <mask> place near the Kilometer Zero marker in the Puerta del Sol in Madrid.0 is the best quality"
get_all_predictions(text_sentence)

that  <mask> place near the Kilometer Zero marker in the Puerta del Sol in Madrid.0 is the best quality


{'bart': 'took\ntakes\nis\ntake\nwould',
 'bert': 'one\ntakes\nspecial\ntook\nlittle',
 'electra': 'in\ntakes\nexact\nthe\none',
 'roberta': 'taking\nmeeting\ntakes\nits\nhis',
 'xlm': 'takes\ntook\ntake\ntaking\nwould',
 'xlnet': 'take\ntakes\ntook\nthey\nthat'}

In [ ]:
text_sentence = "0music is the second album  <mask> with Melomics technology."
get_all_predictions(text_sentence)

0music is the second album  <mask> with Melomics technology.


{'bart': 'recorded\nmade\nproduced\nby\nreleased',
 'bert': 'made\nrecorded\nexperimenting\ndone\nreleased',
 'electra': 'released\nrecorded\npublished\nmade\ndone',
 'roberta': 'made\nproduced\nwritten\nreleased\nrecorded',
 'xlm': 'produced\nreleased\nmade\ndeveloped\ndesigned',
 'xlnet': 'made\nalbum\nwith\nout\nreleased'}

In [ ]:
text_sentence = "<mask> for the operator for trunk calls before subscriber trunk dialing (STD) was  <mask> , and so was  "
get_all_predictions(text_sentence)

<mask> for the operator for trunk calls before subscriber trunk dialing (STD) was  <mask> , and so was  


{'bart': '<s>\nThe\nIt\n</s>\nIn',
 'bert': 'except\naccounting\nonly\nas\nfortunately',
 'electra': 'waiting\ntime\ncall\nonly\nwait',
 'roberta': 'Rules\nRequirements\nCode\nTraining\nApplications',
 'xlm': 'Call\nOffer\nPay\nDemand\nApplication',
 'xlnet': 'was\nand\nmade\ncalled\nbe'}

In [ ]:
text_sentence = "0x10C is a hexadecimal number equivalent to 1612 in decimal, which  <mask> 281,474,976,710,656,"
get_all_predictions(text_sentence)

0x10C is a hexadecimal number equivalent to 1612 in decimal, which  <mask> 281,474,976,710,656,


{'bart': 'has\nequals\nis\nrepresents\ntranslates',
 'bert': 'is\nincludes\nrepresents\ncontains\ncomprises',
 'electra': 'is\nincludes\nmeans\ncomprises\nare',
 'roberta': 'equals\ntotals\nmeasures\nrepresents\nyields',
 'xlm': 'includes\nis\nmeans\ncosts\nequal',
 'xlnet': 'includes\ninclude\nis\nnumber\nwith'}

In [ ]:
text_sentence = "a boxed set <mask> in 2004 on Island Records.$1,000,000 Chance of a Lifetime is an American game show"
get_all_predictions(text_sentence)

a boxed set <mask> in 2004 on Island Records.$1,000,000 Chance of a Lifetime is an American game show


{'bart': 'released\nwas\nof\nfirst\noriginally',
 'bert': 'released\nappeared\nissued\nreissued\npublished',
 'electra': 'released\ndebuted\npremiered\nset\nrecorded',
 'roberta': 'released\nissued\nrelease\npublished\nappeared',
 'xlm': 'released\nappeared\npublished\narrived\nposted',
 'xlnet': 'set\nin\nreleased\nsold\nbox'}

In [ ]:
text_sentence = "1982 SH1, is a Hungaria asteroid from the innermost region of the asteroid belt.100,000 Cobbers is a 1942  <mask> documentary"
get_all_predictions(text_sentence)

1982 SH1, is a Hungaria asteroid from the innermost region of the asteroid belt.100,000 Cobbers is a 1942  <mask> documentary


{'bart': 'Hungarian\nSoviet\nBritish\nBBC\nGerman',
 'bert': 'american\nsoviet\nbritish\nfrench\nshort',
 'electra': 'nasa\namerican\ncanadian\ndocumentary\nbritish',
 'roberta': 'Japanese\nBritish\nGerman\nperiod\nSoviet',
 'xlm': 'Soviet\nfilm\nTV\nhorror\nAmerican',
 'xlnet': 'documentary\ntelevision\nBritish\nAustralian\nfilm'}

In [ ]:
text_sentence = "100.000 dollari per Ringo is a 1965 spaghetti western film <mask> by Alberto De Martino.100000 faction in water is the most attractive, "
get_all_predictions(text_sentence)

100.000 dollari per Ringo is a 1965 spaghetti western film <mask> by Alberto De Martino.100000 faction in water is the most attractive, 


{'bart': 'directed\nstarring\nwritten\nby\nproduced',
 'bert': 'directed\nwritten\nproduced\nmade\ncreated',
 'electra': 'directed\nproduced\nwritten\nmade\nadapted',
 'roberta': 'directed\nnarrated\nproduced\nmade\nadapted',
 'xlm': 'made\ned\nwritten\nplayed\nshot',
 'xlnet': 'directed\nmade\nfilm\nproduced\nwritten'}

In [ ]:
text_sentence = "then mosquitoes are  <mask> instead.100.000 of niets (100,000 or nothing)"
get_all_predictions(text_sentence)

then mosquitoes are  <mask> instead.100.000 of niets (100,000 or nothing)


{'bart': 'eaten\neating\nkilled\nattacked\nbiting',
 'bert': 'killed\ncaught\nused\neaten\nattacked',
 'electra': 'fed\nthere\nintroduced\ncaught\ninfected',
 'roberta': 'killing\nharmless\nkilled\ndying\ndestroying',
 'xlm': 'killed\nborn\ndead\neating\n</s>',
 'xlnet': 'instead\nnot\nthere\nfirst\nthen'}

In [ ]:
text_sentence = "was a Flemish TV quiz ,  <mask> on the Flemish public service TV station N.I.R. (nowadays the VRT), between 1956 and 1959 on Sunday evenings.100,000 (one hundred thousand)"
get_all_predictions(text_sentence)

was a Flemish TV quiz ,  <mask> on the Flemish public service TV station N.I.R. (nowadays the VRT), between 1956 and 1959 on Sunday evenings.100,000 (one hundred thousand)


{'bart': 'shown\nbroadcast\naired\nfirst\noriginally',
 'bert': 'broadcast\nshown\naired\nairing\nrepeated',
 'electra': 'broadcast\naired\nconducted\nairing\nheld',
 'roberta': 'broadcast\nshown\naired\npresented\nscreened',
 'xlm': 'played\nbroadcast\nshown\nshot\nran',
 'xlnet': 'broadcast\naired\nshown\nalso\nrun'}

In [ ]:
text_sentence = "is the natural number  <mask> 99,999 and preceding 100,001.100, 000/-"
get_all_predictions(text_sentence)

is the natural number  <mask> 99,999 and preceding 100,001.100, 000/-


{'bart': 'preceding\nafter\nbetween\nof\nfollowing',
 'bert': 'after\npreceding\nbetween\nfollowing\nbefore',
 'electra': 'preceding\nof\nfollowing\nafter\nabove',
 'roberta': 'of\nabove\nafter\nnumbered\nfor',
 'xlm': 'of\nbeing\nending\nbefore\nsince',
 'xlnet': '99\nand\nof\nnumber\nnow'}

In [ ]:
text_sentence = "(Sri Lankan Rupees) for anyone who  could  <mask> supernatural or miraculous powers under fool-proof and fraud-proof conditions"
get_all_predictions(text_sentence)

(Sri Lankan Rupees) for anyone who  could  <mask> supernatural or miraculous powers under fool-proof and fraud-proof conditions


{'bart': 'demonstrate\nclaim\ndisplay\nprove\nshow',
 'bert': 'demonstrate\nmanifest\nperform\nshow\npossess',
 'electra': 'possess\nhave\nuse\ngain\ndevelop',
 'roberta': 'use\nexercise\npossess\nclaim\nwield',
 'xlm': 'escape\npossess\nmaster\nabsorb\nuse',
 'xlnet': 'supernatural\nhave\nperform\nmiraculous\nmake'}

In [ ]:
text_sentence = "Gujarat Police.10,000 acres (40 km2) of  <mask> > land"
get_all_predictions(text_sentence)

Gujarat Police.10,000 acres (40 km2) of  <mask> > land


{'bart': 'forest\ntheir\nits\ngovernment\npolice',
 'bert': 'normal\nflat\nresource\ntrust\nvacant',
 'electra': 'land\nforest\npublic\ntar\na',
 'roberta': 'forest\nnon\nagricultural\nforestry\nforests',
 'xlm': 'land\nretail\nindustrial\ntotal\nproperty',
 'xlnet': 'of\nthe\nits\nall\nhis'}

In [ ]:
text_sentence = "Black Men Named George is "
get_all_predictions(text_sentence)

Black Men Named George is 


IndexError: ignored

In [ ]:
text_sentence = "Ranchers Bees Stadium is their home venue.10,000 copies were  <mask> "
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "1.10,000 km is a Spanish romantic drama film  <mask> by Carlos Marques-Marcet."
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Zeni Geva.10.000 luchtballonnen is the fourteenth studio album by Belgian-Dutch girl group K3.10,000 Maniacs in 2001.It was quickly a sextet, which  <mask> Ryan Seekings on drums, Stan Barton on fiddle,"
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "10,000 Maniacs is an American alternative rock band that was  <mask> in 1981.10,000 maps 4,000 musical scriptsThe research library today has approximately 850,000 volumes with collection emphasis on the German literature."
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The research library today has approximately 850,000 volumes with collection emphasis on the German literature.10000 mEach event is <mask> according to the decathlon  <mask> tables or, for non-decathlon events, "
get_all_predictions(text_sentence)